In [4]:
import numpy as np
import pandas as pd

# Definir parâmetros específicos para o teste
s_0 = 0
m = 5  # kg
k = 0.5  # coeficiente de arrasto
t = 5  # segundos
g = 9.81  # aceleração da gravidade em m/s^2

# Calcular a velocidade e a posição usando a fórmula
v_analitica = (m * g / k) * (1 - np.exp(-k * t / m))
y_analitica = (m * g / k) * t - (m**2 * g / k**2) * (1 - np.exp(-k * t / m))

print(f'Velocidade analítica: {v_analitica:.2f} m/s')
print(f'Posição analítica: {y_analitica:.2f} m')

Velocidade analítica: 38.60 m/s
Posição analítica: 104.51 m


In [7]:

# Definindo constantes
g = 9.81  # aceleração da gravidade (m/s^2)

# Geração de dados
np.random.seed(42)
n_samples = 1000

massas = np.random.uniform(1, 10, n_samples)  # massa entre 1 e 10 kg
k_arrastos = np.random.uniform(0.1, 1, n_samples)  # constante de arrasto entre 0.1 e 1
tempos = np.random.uniform(0, 10, n_samples)  # tempo entre 0 e 10 segundos

velocidades = []
posicoes = []

for m, k, t, y0 in zip(massas, k_arrastos, tempos):
    v_t = (m * g / k) * (1 - np.exp(-k * t / m))
    y_t = (m * g / k) * t - (m**2 * g / k**2) * (1 - np.exp(-k * t / m))
    velocidades.append(v_t)
    posicoes.append(y_t)

# Criação do DataFrame
df = pd.DataFrame({
    'massa': massas,
    'constante_de_arrasto': k_arrastos,
    'tempo': tempos,
    'velocidade': velocidades,
    'posicao': posicoes
})
# Salvar em um arquivo CSV
df.to_csv('queda_objeto.csv', index=False)


ValueError: not enough values to unpack (expected 4, got 3)

In [8]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Carregar os dados do CSV
df = pd.read_csv('queda_objeto.csv')

# Separar as entradas (X) e as saídas (y)
X = df[['massa', 'constante_de_arrasto', 'tempo']]
y = df[['velocidade', 'posicao']]

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Padronizar os dados
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)
y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)


In [9]:
# Construir a rede neural
model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),
    Dense(64, activation='relu'),
    Dense(2, activation='linear')  # Saída de 2 dimensões: velocidade e posição
])

# Compilar o modelo
model.compile(optimizer='adam', loss='mse', metrics=['mae'])


c:\GitHub\ProjetosPessoais\redeNeuralArtificial\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
# Treinar o modelo
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)


Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.7815 - mae: 0.7172 - val_loss: 0.3638 - val_mae: 0.4854
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2932 - mae: 0.4107 - val_loss: 0.0971 - val_mae: 0.2246
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0843 - mae: 0.2095 - val_loss: 0.0482 - val_mae: 0.1629
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0320 - mae: 0.1386 - val_loss: 0.0299 - val_mae: 0.1255
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0235 - mae: 0.1178 - val_loss: 0.0234 - val_mae: 0.1086
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0166 - mae: 0.0992 - val_loss: 0.0196 - val_mae: 0.0972
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0132 - mae: 0.0858 - val_loss: 0.0175 - val_mae: 0.0903
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0122 - mae: 0.0823 - val_loss: 0.0151 - val_mae: 0.0849
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.00

In [11]:
# Avaliar o modelo
loss, mae = model.evaluate(X_test, y_test)
print(f'Mean Absolute Error on test data: {mae}')

# Salvar o modelo treinado
model.save('queda_objeto.keras')


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0011 - mae: 0.0212  
Mean Absolute Error on test data: 0.021667804569005966


In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import math

# Carregar o modelo treinado
model = load_model('queda_objeto.keras')

# Definir parâmetros específicos para o teste
m = 5  # kg
k = 0.5  # coeficiente de arrasto
t = 5  # segundos
g = 9.81  # aceleração da gravidade em m/s^2

# Calcular a velocidade e a posição usando a fórmula analítica
v_analitica = (m * g / k) * (1 - np.exp(-k * t / m))
y_analitica = (m * g / k) * t - (m**2 * g / k**2) * (1 - np.exp(-k * t / m))

# Preparar os dados de entrada para a rede neural
entrada = np.array([[m, k, t]])
entrada_padronizada = scaler_X.transform(entrada)

# Fazer a previsão usando a rede neural
saida_predita_padronizada = model.predict(entrada_padronizada)
saida_predita = scaler_y.inverse_transform(saida_predita_padronizada)

v_predita, y_predita = saida_predita[0]

# Exibir os resultados
print(f"Resultados Formula: Velocidade = {v_analitica:.4f} m/s, Posição = {y_analitica:.4f} m")
print(f"Previsões da Rede Neural: Velocidade = {v_predita:.4f} m/s, Posição = {y_predita:.4f} m")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Resultados Formula: Velocidade = 38.5993 m/s, Posição = 104.5066 m
Previsões da Rede Neural: Velocidade = 38.4627 m/s, Posição = 102.8974 m


c:\GitHub\ProjetosPessoais\redeNeuralArtificial\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
